<a href="https://colab.research.google.com/github/Nonnyss/Ms-Wav2Vec2-Finetune/blob/main/Music_wav2vec2_thai_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##gpu_check

In [1]:
!nvidia-smi

Tue Jun 21 16:55:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Huggingface-Login

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#git config --global credential.helper store

In [ ]:
#huggingface-cli login

## Installs and Imports



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt-get install --reinstall build-essential --yes

In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datasets==1.11.0
!pip install transformers==4.9.1
!pip install jiwer
!pip install ipywidgets
!pip install pythainlp==2.3.1

In [5]:
from functools import partial
import pandas as pd
import numpy as np
from datasets import (
    load_dataset, 
    load_from_disk,
    load_metric,)
from transformers import (
    Wav2Vec2CTCTokenizer, 
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
)
import torchaudio
import re
import json
from pythainlp.tokenize import word_tokenize, syllable_tokenize


## Data Preparation

In [ ]:
#!unzip /content/drive/MyDrive/Main_Data/Data1.zip -d /content/drive/MyDrive/Main_Data

In [6]:
datasets = load_dataset("/wav2vec2-large-xlsr-53-th-main/th_common_voice_70.py")
datasets

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/th/7.0.0/b487215783843e604168505e26993c1b09bd225a366b11f4875e530cfc2ed5ee. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['path', 'sentence'],
        num_rows: 670
    })
    test: Dataset({
        features: ['path', 'sentence'],
        num_rows: 19
    })
    validation: Dataset({
        features: ['path', 'sentence'],
        num_rows: 269
    })
})

In [7]:
def preprocess_data(example, tok_func = word_tokenize):
    example['sentence'] = ' '.join(tok_func(example['sentence']))
    return example

datasets = datasets.map(preprocess_data)

Parameter 'function'=<function preprocess_data at 0x7fb4c53b3290> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/670 [00:00<?, ?ex/s]

  0%|          | 0/19 [00:00<?, ?ex/s]

  0%|          | 0/269 [00:00<?, ?ex/s]

### Exploration

#### `sentence`; transcripts

In [ ]:
#show random sentences
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    
show_random_elements(datasets["train"].remove_columns(["path"]), num_examples=20)

,sentence
0,และ รู้ เอาไว้ ว่า
1,กว่า เป็นเพื่อน กัน
2,เลิก แสดง
3,ก่อน ดี ไหม
4,และ ฉัน ก็ คิด อยาก มีสิทธิ์ มากกว่า นั้น
5,หาก เขา ได้ ทำให้ เธอ ปวดร้าว มอง มา ที่ เดิม เดิม จะ เห็น คน เดิม คน นี้
6,อยาก ขอ
7,คอย เฝ้าถาม และ ห่วงใย
8,ให้ คำ ว่า เรา
9,ยังคง เป็น อย่าง เดิม อย่างนี้ เรื่อยไป


## Tokenizer & Feature

#### `Non` Tokenizer

In [ ]:
def extract_all_chars(batch, text_col = "sentence"):
    all_text = " ".join(batch[text_col])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = datasets.map(extract_all_chars, 
                   batched=True, 
                   batch_size=-1, 
                   keep_in_memory=True, 
                   remove_columns=datasets.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["validation"]["vocab"][0]) |set(vocabs["test"]["vocab"][0]))
# vocab_list = list(set(vocabs["train"]["vocab"][0])) #strictly no leakage
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
len(vocab_dict), vocab_dict

In [ ]:
#make space = |
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
#padding token serves as blank token
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict), vocab_dict

In [ ]:
 #save as json; create tokenizer for the first time and upload to hugginface hub
 with open('/content/drive/MyDrive/save-pretrain/vocab.json', 'w') as vocab_file:
     json.dump(vocab_dict, vocab_file)
 tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/save-pretrain/vocab.json", 
                                  unk_token="[UNK]", 
                                  pad_token="[PAD]", 
                                  word_delimiter_token="|")
 tokenizer.save_pretrained('/content/drive/MyDrive/save-pretrain/Token')

('/content/drive/MyDrive/save-pretrain/Token/tokenizer_config.json',
 '/content/drive/MyDrive/save-pretrain/Token/special_tokens_map.json',
 '/content/drive/MyDrive/save-pretrain/Token/vocab.json',
 '/content/drive/MyDrive/save-pretrain/Token/added_tokens.json')

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/save-pretrain/Token")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### `Aibuilder` Tokenizer

In [14]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
datasets['train'][0]['sentence']

'แค่ อยาก จะ ขอ เก็บ ไว้ ที่ เก่า'

In [ ]:
tokenizer(datasets['train'][0]['sentence'])

{'input_ids': [50, 46, 15, 42, 19, 24, 16, 2, 42, 49, 5, 42, 58, 19, 42, 28, 2, 25, 33, 42, 59, 47, 7, 42, 21, 32, 15, 42, 28, 2, 15, 16], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.decode(tokenizer(datasets['train'][0]['sentence']).input_ids)

'แค่ อยาก จะ ขอ เก็บ ไว้ ที่ เก่า'

In [ ]:
tokenizer.vocab_size

70

#### `Non` Feature

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, 
                                             sampling_rate=16000, 
                                             padding_value=0.0, 
                                             do_normalize=True, 
                                             return_attention_mask=False)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/save-pretrain/Processor")

#### `Aibuilder` Feature

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

In [15]:
processor = Wav2Vec2Processor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Preprocess Data

In [8]:
def speech_file_to_array_fn(batch, 
                            text_col="sentence", 
                            fname_col="path",
                            resampling_to=16000):
    speech_array, sampling_rate = torchaudio.load(batch[fname_col])
    resampler=torchaudio.transforms.Resample(sampling_rate, resampling_to)
    batch["speech"] = resampler(speech_array)[0].numpy()
    batch["sampling_rate"] = resampling_to
    batch["target_text"] = batch[text_col]
    return batch

In [9]:
speech_datasets = datasets.map(speech_file_to_array_fn, 
                                   remove_columns=datasets.column_names["train"])
speech_datasets

  0%|          | 0/670 [00:00<?, ?ex/s]

  0%|          | 0/19 [00:00<?, ?ex/s]

  0%|          | 0/269 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 670
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 19
    })
    validation: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 269
    })
})

In [10]:
#sample sounds
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(speech_datasets["train"]))
print(speech_datasets["train"][rand_int]["target_text"])
ipd.Audio(data=np.asarray(speech_datasets["train"][rand_int]["speech"]), autoplay=True, rate=16000)

แต่ เธอ ยัง รัก อยู่ เรื่อย มา


In [11]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [15]:
prepared_datasets = speech_datasets.map(prepare_dataset, 
                                        remove_columns=speech_datasets.column_names["train"], 
                                        batch_size=16,
                                        batched=True)

  0%|          | 0/42 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [16]:
prepared_datasets

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 670
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 19
    })
    validation: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 269
    })
})

## Training

#### `Data` collator

In [17]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [18]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

#### `Metric` wer

In [6]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [44]:
wer_metric.compute(predictions=['ลม สก โรงไหล ฉัน'],references=['ล้ม ซบ ตรง ไหล่ ฉัน'])

0.8

In [20]:
def compute_metrics(pred, processor, metric):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

#### `Model` train

In [21]:
torch.cuda.is_available()

True

In [22]:
model = Wav2Vec2ForCTC.from_pretrained(
    "airesearch/wav2vec2-large-xlsr-53-th",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [23]:
model.freeze_feature_extractor()

In [24]:
training_args = TrainingArguments(
    output_dir="/Music-wav2vec2-finetune",
    group_by_length=True,
    push_to_hub=True,
    fp16=True,
    per_device_train_batch_size=24,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=12,
    metric_for_best_model='wer',
    evaluation_strategy="steps",
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=40,
    num_train_epochs=50,
    learning_rate=2e-4,
    warmup_steps=500,
    save_total_limit=2,
    report_to="tensorboard"
)

    #per_device_train_batch_size=16,     for p100
    #gradient_accumulation_steps=1,
    #per_device_eval_batch_size=8,

In [25]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=partial(compute_metrics, metric=wer_metric, processor=processor),
    train_dataset=prepared_datasets["train"],
    eval_dataset=prepared_datasets["validation"],
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [ ]:
trainer.train()

## Inference and Evaluation

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("Nonnyss/music-wav2vec2-th-finetune")

In [21]:
test_dataset = load_dataset("/content/drive/MyDrive/wav2vec2-large-xlsr-53-th-main/th_common_voice_70.py", split="test")
test_dataset

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/th/7.0.0/b487215783843e604168505e26993c1b09bd225a366b11f4875e530cfc2ed5ee. Subsequent calls will reuse this data.


Dataset({
    features: ['path', 'sentence'],
    num_rows: 19
})

In [29]:
def speech_file_to_array_fn(batch, 
                            text_col="sentence", 
                            fname_col="path",
                            resampling_to=16000):
    speech_array, sampling_rate = torchaudio.load(batch[fname_col])
    resampler=torchaudio.transforms.Resample(sampling_rate, resampling_to)
    batch["speech"] = resampler(speech_array)[0].numpy()
    batch["sampling_rate"] = resampling_to
    batch["target_text"] = batch[text_col]
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
print(test_dataset["speech"][:2])
inputs = processor(test_dataset["speech"][:5], sampling_rate=16_000, return_tensors="pt", padding=True)

  0%|          | 0/19 [00:00<?, ?ex/s]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.960464477539063e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.960464477539063e-08, -5.960464477539063e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.960464477539063e-08, 5.960464477539063e-08, 0.0, 0.0, -5.960464477539063e-08, 0.0, 0.0, -5.960464477539063e-08, 0.0, 0.0, 0.0, 0.0, -5.960464477539063e-08, -5.960464477539063e-08, -1.7881393432617188e-07, 0.0, 5.960464477539063e-08, -5.960464477539063e-08, 5.960464477539063e-08, 0.0, 5.960464477539063e-08, 5.960464477539063e-08, -5.960464477539063e-08, 5.960464477539063e-08, 0.0, -5.960464477539063e-08, 1.1920928955078125e-07, 3.5762786865234375e-07, 1.1920928955078125e-07, -1.1920928955078125e-07, -5.960464477539063e-08, -2.3841

In [30]:
import torch
with torch.no_grad():
    logits = model(inputs.input_values,).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][6:11])

Prediction: ['ทำไม อยู่ ตรงนี้', 'ลม สก โรงไหล ฉัน', 'เดือน ครั้งแรก ที่ เธอ ได้ พบ รัก', 'มี เรื่อง อืดอาบ มิดใจ', 'ที่ เธอ ได้ ไป จาก ฉัน']
Reference: ['ทำไมอยู่ตรงนี้', 'ล้มซบตรงไหล่ฉัน', 'เหมือนครั้งแรกที่เธอได้พบรัก', 'มีเรื่องอึดอัดในใจ', 'ที่เธอได้ไปจากฉัน']
